In [1]:
# use ansible to let argocd setup platform (minio, postgres, mlflow)

In [2]:
export PATH=/work/.local/bin:$PATH
export PYTHONUSERBASE=/work/.local
export ANSIBLE_CONFIG=/work/MLSysEngOps-NYU25Spring/ansible/ansible.cfg
export ANSIBLE_ROLES_PATH=roles

In [3]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/argocd_add_platform2.yml


PLAY [Mount persistent volume and deploy MLflow platform with MinIO] ***********

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Detect filesystem on /dev/vdb] *******************************************
ok: [node1]

TASK [Format /dev/vdb as ext4 on first use] ************************************
skipping: [node1]

TASK [Make root mount directory] ***********************************************
changed: [node1]

TASK [Mount volume & ensure /etc/fstab] ****************************************
changed: [node1]

TASK [Make MinIO data dir] *****************************************************
ok: [node1]

TASK [Make Postgres data dir] **************************************************
ok: [node1]

PLAY [Deploy MLflow platform via ArgoCD & Helm with MinIO secret handling] *****

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Get ArgoCD admin password from Kubernetes secret] ***********

In [4]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/workflow_build_init.yml


PLAY [Run Argo Workflow from GitHub Repo] **************************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Clone or update project37 repo] ******************************************
changed: [node1]

TASK [Submit Argo Workflow] ****************************************************
changed: [node1]

TASK [Extract Workflow Name] ***************************************************
ok: [node1]

TASK [Wait for workflow to complete (success or fail)] *************************
changed: [node1]

TASK [Get final workflow result] ***********************************************
changed: [node1]

TASK [Display workflow phase] **************************************************
ok: [node1] => 
  msg: 'Workflow kaniko-pvc-build-5jwtv finished with status: Succeeded'

TASK [Fail if workflow did not succeed] ****************************************
skipping: [node1]

PLAY RECAP ***************************************************

In [5]:
cd /work/MLSysEngOps-NYU25Spring/k8s/staging/
cat templates/project37-app.yaml

# Deployment
apiVersion: apps/v1
kind: Deployment
metadata:
  name: project37-app
  namespace: project37-staging

spec:
  replicas: {{ .Values.replicaCount }}
  selector:
    matchLabels:
      app: project37-app
  template:
    metadata:
      labels:
        app: project37-app
    spec:
      containers:
        - name: project37-app
          image: {{ .Values.image.repository }}:{{ .Values.image.tag }}
          ports:
            - containerPort: 3000
            - containerPort: 8000
          resources:
            {{- toYaml .Values.resources | nindent 12 }}
---
# Service
apiVersion: v1
kind: Service
metadata:
  name: project37-app
  namespace: project37-staging
spec:
  selector:
    app: project37-app
  ports:
    - name: web
      port: {{ .Values.service.port }}      
      targetPort: 3000
    - name: api                             
      port: 8001                            
      targetPort: 8000  
  externalIPs:
    - {{ .Values.service.externalIP }}



In [6]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/argocd_add_staging.yml


PLAY [Deploy Project37 Staging via ArgoCD & Helm] ******************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Get ArgoCD admin password] ***********************************************
changed: [node1]

TASK [Decode ArgoCD password] **************************************************
changed: [node1]

TASK [Login to ArgoCD] *********************************************************
ok: [node1]

TASK [Detect external IP starting with 10.56] **********************************
ok: [node1]

TASK [Create project37-staging namespace if missing] ***************************
changed: [node1]

TASK [Check if ArgoCD app exists] **********************************************
ok: [node1]

TASK [Create ArgoCD Helm app if not exists] ************************************
changed: [node1]

TASK [Update ArgoCD Helm app if exists] ****************************************
skipping: [node1]

TASK [Display ArgoCD app status] ************

In [7]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/argocd_add_canary2.yml


PLAY [Deploy Project37 Canary via ArgoCD & Helm] *******************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Get ArgoCD admin password] ***********************************************
changed: [node1]

TASK [Decode ArgoCD password] **************************************************
changed: [node1]

TASK [Login to ArgoCD] *********************************************************
ok: [node1]

TASK [Detect external IP starting with 10.56] **********************************
ok: [node1]

TASK [Create project37-canary namespace if missing] ****************************
changed: [node1]

TASK [Check if ArgoCD app exists] **********************************************
ok: [node1]

TASK [Create ArgoCD Helm app if not exists] ************************************
changed: [node1]

TASK [Update ArgoCD Helm app if exists] ****************************************
skipping: [node1]

TASK [Display ArgoCD app status] ************

In [8]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/argocd_add_prod.yml


PLAY [Deploy Project37 Production via ArgoCD & Helm] ***************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Get ArgoCD admin password] ***********************************************
changed: [node1]

TASK [Decode ArgoCD password] **************************************************
changed: [node1]

TASK [Login to ArgoCD] *********************************************************
ok: [node1]

TASK [Detect external IP starting with 10.56] **********************************
ok: [node1]

TASK [Create project37-production namespace if missing] ************************
changed: [node1]

TASK [Check if ArgoCD app exists] **********************************************
ok: [node1]

TASK [Create ArgoCD Helm app if not exists] ************************************
changed: [node1]

TASK [Update ArgoCD Helm app if exists] ****************************************
skipping: [node1]

TASK [Display ArgoCD app status] ************

In [9]:
# go to project_lifecycles.ipynb